## script version

# notebook version

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pd.options.display.max_columns = 999
# pd.options.display.max_rows = 999
pd.options.display.max_seq_items = None
# np.set_printoptions(threshold=100)

In [5]:
import sys
import os
import pandas as pd
from tqdm import tqdm
sys.path.append(os.path.join(os.path.expanduser('~'), 'projects'))
from j_utils import munging as mg
from j_utils import eval_results as er



## old version

In [2]:
import sys
import os
import pandas as pd

sys.path.append(os.path.join(os.path.expanduser('~')))
from tqdm import tqdm
from j_utils import munging as mg
from j_utils import eval_results as er
# from fastai.structured import *

pd.options.display.max_columns = 999
# pd.options.display.max_rows = 999
pd.options.display.max_seq_items = None
# np.set_printoptions(threshold=100)

ModuleNotFoundError: No module named 'j_utils'

In [3]:
PATH = dc.data_path
project = 'lendingclub'
fname_base_loan = 'base_loan_info.fth'
fname_eval_loan = 'eval_loan_info.fth'
fname_str_loan = 'str_loan_info.fth'
fname_scl_pmt = 'scaled_pmt_hist.fth'
data_path = f'{PATH}{project}'

In [4]:
pmt_hist = pd.read_feather(f'{data_path}/scaled_pmt_hist.fth')

In [9]:
eval_loan_info = pd.read_feather(f'{data_path}/eval_loan_info.fth')

In [5]:
interesting_cols_over_time = [
    'outs_princp_beg',
    'all_cash_to_inv',
    'date',
    'fico_last',
    'm_on_books',
    'status_period_end',
    'id',
]
pmt_hist = pmt_hist[interesting_cols_over_time]

In [6]:
npv_roi_holder = {}
disc_rates = np.arange(.05,.36,.01)
id_grouped = pmt_hist.groupby('id')
for ids, group in tqdm(id_grouped):
    npv_roi_dict = {}
    funded = group.iat[0,0]
    cfs = [-funded] + group['all_cash_to_inv'].tolist()
    for rate in disc_rates:
        npv_roi_dict[rate] = np.npv(rate/12, cfs)/funded
    npv_roi_holder[ids] = npv_roi_dict

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/justin/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/justin/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 109, in run
    if instance.miniters > 1 and \
AttributeError: 'tqdm' object has no attribute 'miniters'

100%|██████████| 2003523/2003523 [19:36<00:00, 1703.23it/s]


In [13]:
npv_roi_df = pd.DataFrame(npv_roi_holder).T
npv_roi_df.columns = npv_roi_df.columns.values.round(2)
npv_roi_df.index.name = 'id'
npv_roi_df.reset_index(inplace=True)

In [18]:
eval_rows = eval_loan_info.shape[0]

In [19]:
eval_rows

2003915

In [21]:
eval_loan_info = pd.merge(eval_loan_info, npv_roi_df, how='left', on='id')

In [23]:
for col in eval_loan_info.columns:
    if len(eval_loan_info[eval_loan_info[col].isnull()]) > 0:
        print(col)

target_strict
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35


In [26]:
eval_loan_info[eval_loan_info['issue_d'] == eval_loan_info['issue_d'].max()]

,end_d,issue_d,maturity_paid,maturity_time,maturity_time_stat_adj,maturity_paid_stat_adj,rem_to_be_paid,roi_simple,target_loose,target_strict,loan_status,id,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.3,0.31,0.32,0.33,0.34,0.35
1798332,2018-07-01,2018-06-01,1.000000,0.033333,0.033333,1.000000,0.000000,1.007832,0,0.0,paid,127549768,0.004207,0.003375,0.002543,0.001713,0.000885,0.000058,-0.000768,-0.001593,-0.002416,-0.003237,-0.004058,-0.004877,-0.005694,-0.006511,-0.007326,-0.008140,-0.008952,-0.009763,-0.010573,-0.011381,-0.012188,-0.012994,-0.013798,-0.014601,-0.015403,-0.016203,-0.017003,-0.017800,-0.018597,-0.019392,-0.020186
1813920,2018-08-01,2018-06-01,0.056292,0.055556,0.055556,0.056292,44556.852362,0.066446,0,0.0,current,128253069,-0.965845,-0.965873,-0.965902,-0.965930,-0.965958,-0.965986,-0.966014,-0.966042,-0.966070,-0.966098,-0.966126,-0.966154,-0.966182,-0.966210,-0.966237,-0.966265,-0.966293,-0.966320,-0.966348,-0.966375,-0.966403,-0.966430,-0.966457,-0.966485,-0.966512,-0.966539,-0.966566,-0.966594,-0.966621,-0.966648,-0.966675
1840903,2018-08-01,2018-06-01,0.054818,0.055556,0.055556,0.054818,7421.188203,0.071735,0,0.0,current,129431896,-0.963771,-0.963801,-0.963831,-0.963861,-0.963891,-0.963921,-0.963950,-0.963980,-0.964010,-0.964040,-0.964069,-0.964099,-0.964128,-0.964158,-0.964187,-0.964216,-0.964246,-0.964275,-0.964304,-0.964333,-0.964362,-0.964392,-0.964421,-0.964450,-0.964478,-0.964507,-0.964536,-0.964565,-0.964594,-0.964622,-0.964651
1843731,2018-08-01,2018-06-01,0.032307,0.033333,0.033333,0.032307,31715.012785,0.044117,0,0.0,current,129555197,-0.977310,-0.977329,-0.977348,-0.977367,-0.977385,-0.977404,-0.977423,-0.977441,-0.977460,-0.977479,-0.977497,-0.977516,-0.977534,-0.977553,-0.977571,-0.977589,-0.977608,-0.977626,-0.977644,-0.977663,-0.977681,-0.977699,-0.977717,-0.977735,-0.977753,-0.977772,-0.977790,-0.977808,-0.977826,-0.977844,-0.977862
1848453,2018-08-01,2018-06-01,0.033000,0.033333,0.033333,0.033000,30188.555647,0.039624,0,0.0,current,129749791,-0.980064,-0.980080,-0.980097,-0.980113,-0.980130,-0.980146,-0.980163,-0.980179,-0.980195,-0.980212,-0.980228,-0.980244,-0.980260,-0.980277,-0.980293,-0.980309,-0.980325,-0.980341,-0.980357,-0.980373,-0.980389,-0.980405,-0.980421,-0.980437,-0.980453,-0.980469,-0.980485,-0.980501,-0.980517,-0.980532,-0.980548
1854172,2018-08-01,2018-06-01,0.032424,0.033333,0.033333,0.032424,25250.135804,0.042307,0,0.0,current,129955243,-0.978323,-0.978341,-0.978359,-0.978377,-0.978395,-0.978412,-0.978430,-0.978448,-0.978466,-0.978484,-0.978501,-0.978519,-0.978537,-0.978554,-0.978572,-0.978589,-0.978607,-0.978624,-0.978642,-0.978659,-0.978677,-0.978694,-0.978711,-0.978729,-0.978746,-0.978763,-0.978781,-0.978798,-0.978815,-0.978832,-0.978849
1862847,2018-08-01,2018-06-01,0.032689,0.033333,0.033333,0.032689,41637.497088,0.050253,0,0.0,current,130299207,-0.974467,-0.974489,-0.974510,-0.974531,-0.974552,-0.974573,-0.974594,-0.974615,-0.974636,-0.974657,-0.974678,-0.974698,-0.974719,-0.974740,-0.974761,-0.974781,-0.974802,-0.974823,-0.974843,-0.974864,-0.974884,-0.974905,-0.974925,-0.974946,-0.974966,-0.974986,-0.975007,-0.975027,-0.975047,-0.975067,-0.975088
1870788,2018-08-01,2018-06-01,0.032782,0.033333,0.033333,0.032782,19113.400767,0.046272,0,0.0,current,130664455,-0.976558,-0.976578,-0.976597,-0.976617,-0.976636,-0.976655,-0.976675,-0.976694,-0.976713,-0.976732,-0.976751,-0.976770,-0.976790,-0.976809,-0.976828,-0.976847,-0.976866,-0.976884,-0.976903,-0.976922,-0.976941,-0.976960,-0.976979,-0.976997,-0.977016,-0.977035,-0.977054,-0.977072,-0.977091,-0.977109,-0.977128
1874820,2018-08-01,2018-06-01,0.032846,0.033333,0.033333,0.032846,25362.232559,0.043946,0,0.0,current,130827341,-0.977782,-0.977801,-0.977819,-0.977838,-0.977856,-0.977874,-0.977892,-0.977911,-0.977929,-0.977947,-0.977965,-0.977983,-0.978001,-0.978020,-0.978038,-0.978056,-0.978074,-0.978091,-0.978109,-0.978127,-0.978145,-0.9781

In [28]:
eval_loan_info['target_strict'] = eval_loan_info['target_strict'].fillna(0)
eval_loan_info.fillna(-1, inplace=True)

In [32]:
# feather must have string column names
eval_loan_info.columns = [str(col) for col in eval_loan_info.columns]

In [33]:
eval_loan_info.to_feather(f'{data_path}/eval_loan_info.fth')

In [ ]:
# some setups for code block below
pmt_hist_ids = store['pmt_hist_ids'].astype(int)
max_id = pmt_hist_ids.max()
chunksize = 800
n_chunks = len(pmt_hist_ids)//chunksize + 1

interesting_cols_over_time = [
    'outs_princp_beg',
    'all_cash_to_inv',
    'date',
    'fico_last',
    'm_on_books',
    'status_period_end',
    'loan_id',
    'loan_id_num',
]

disc_rates = np.arange(.05,.36,.01)
# printed_cols = set()
# npv_roi_holder = {}
# for n in tqdm_notebook(np.arange(n_chunks)):
#     if n == 0:
#         left_bound = 0
#     else:
#         left_bound = pmt_hist_ids[n*chunksize]
#     if n == (n_chunks - 1):
#         right_bound = max_id
#     else:
#         right_bound = pmt_hist_ids[(n+1)*chunksize]
    
#     chunk = pd.read_hdf(
#         store,
#         'pmt_hist_intermediary_2',
#         where='(loan_id_num > left_bound) & (loan_id_num <= right_bound)',
#         columns = interesting_cols_over_time)
    
#     id_grouped = chunk.groupby('loan_id')
#     for ids, group in id_grouped:
#         npv_roi_dict = {}
#         funded = group.iat[0,0]
#         cfs = [-funded] + group['all_cash_to_inv'].tolist()
#         for rate in disc_rates:
#             npv_roi_dict[rate] = np.npv(rate/12, cfs)/funded
#         npv_roi_holder[ids] = npv_roi_dict

In [30]:
# store['loan_npv_rois'] = npv_roi_df
# store.close()